<h1> Analizator Logów + Reguły Asocjacyjne</h1>

<h2>Implementacja tworzenia sesji na podstawie żądań bez użycia zewnętrznych bibliotek</h2>

In [25]:
import re # Import biblioteki do Regex'ów.

In [26]:
def get_requests(path_to_log_file):
    """
    Uzyskanie listy wszystkich żądań.
    Args:
        path_to_log_file (string): Ścieżka do pliku z logami

    Returns:
        requests (list): Lista zawierająca wszystkie żądania.
    """
    log_file = open(f"{path_to_log_file}", 'r')
    patterns = {
        "ip_address": '(\d+.\d+.\d+.\d+)\s-\s-\s',
        "data_time": '\[(.+)\]\s',
        "requested_file": '"GET\s(.+)\s\w+/.+"\s\d+\s',
        "status_code":'\d{3}' 
    }
    requests = []
    line_id = 0
    for line in log_file.readlines():
        line = line.replace("black", "-")
        line = line.replace("POST", "GET")
        line = line.replace("HEAD", "GET")
        line_id +=1
        try:
            result = []
            result.append(re.findall(patterns["ip_address"], line)[0])
            result.append(re.findall(patterns["data_time"], line)[0])
            result.append(re.findall(patterns["requested_file"], line)[0])
            result.append(re.findall(patterns["status_code"], line)[0])
            requests.append(result)
        except:
            print(f"DEBUG:: {line}")
            print(f"DEBUG:: LINE ID: {line_id}")
            assert False
    return requests

In [27]:
def get_requests_for_user(ip_address, requests):
    """
    Uzyskanie wszystkich żądań dotyczącego danego hosta.
    Args:
        ip_address (string): Adres IP danego urządzenia sieciowego.
        requests (list): Lista/ tablica żądań.

    Returns:
        all_user_requests: Wszystkie żądania dotyczącego określonego hosta.
    """
    results:list = []
    for record in requests:
        if record[0] == ip_address:
            results.append(record)
    return results

def get_sessions_for_user(ip_address, requests, max_time_in_seconds=20, log_debug = True):
    """
    Uzyskanie sesji dotyczących określonego urządzenie sieciowego/ hosta.
    Args:
        ip_address (string): Adres ip dotyczącego danego hosta.
        requests (list): Lista/ macierz żądań.
        max_time_in_seconds (int): Maksymalny zakres trwania pojedynczej sesji.
        log_debug (bool): Wartość boolowska służąca do pomocy przy debugowaniu.

    Returns:
       all_user_sessions: Wszystkie utworzone sesje przez użytkownika sieciowego.
    """
    user_requests = get_requests_for_user(ip_address, requests)
    grouped_sessions = [[]]
    first_record = True
    session_id = 0
    last_time_in_seconds = 0
    for record in user_requests:
        formatted_time = str(record[1]).split(" ")[0].split("/")[2].split(":")
        current_time_in_seconds = int(formatted_time[2])*60 + int(formatted_time[3])
        
        if first_record:
            last_time_in_seconds = current_time_in_seconds
            first_record = False
            pass
        
        delta_time_in_seconds = abs(int(current_time_in_seconds) - int(last_time_in_seconds))
        
        if log_debug:
            print(f"------------")
            print(f"Last time in seconds: {last_time_in_seconds}")
            print(f"Current time in seconds: {current_time_in_seconds}")
            print(f"Delta time in seconds: {delta_time_in_seconds}")
            print(f"------------")
        
        if delta_time_in_seconds < max_time_in_seconds:
            grouped_sessions[session_id].append(record)
        else:
            session_id+=1
            grouped_sessions.append([])
            grouped_sessions[session_id].append(record)
    return grouped_sessions
        
def get_ip_addresses(requests):
    """
    Uzyskanie listy zawierającej wszystkie adresy IP urządzeń sieciowych.
    Args:
        requests (lista): Lista zawierająca wszystkie żądania.

    Returns:
        ip_addresses: Lista zawierająca wszystkie adresy sieciowe urządzeń.
    """
    ip_addresses = set()
    for record in requests:
        ip_addresses.add(record[0])
    return list(ip_addresses)

def get_sessions(path_to_log_file, max_time_in_seconds=20, log_debug=False):
    """
    Uzyskanie wszystkich sesji dla każdego adresu IP.
    Args:
        max_time_in_seconds (float): Maksymalna wartość czasowa sesji.
        path_to_log_file (string) : Ścieżka do pliku z logami.
        log_debug (bool): Wartość boolowska dotycząca debugowania/ drukowania pomocniczych komunikatów.

    Returns:
        _type_: _description_
    """
    requests = get_requests(path_to_log_file=path_to_log_file) # Uzyskanie wszystkich żądań w postaci macierzy (lista tablic).
    ip_addreses = get_ip_addresses(requests) # Uzyskanie listy wszystkich adresów IP.
    sessions = {} # Pusty słownik dotyczący adresów IP oraz ich sesji.
    for ip_address in ip_addreses:
        if log_debug:
            print(f"Current ip address: {ip_address}") # Uzyskiwanie sesji dla każdego adresu IP
        sessions[ip_address]=get_sessions_for_user(ip_address=ip_address, requests=requests, max_time_in_seconds=max_time_in_seconds, log_debug=log_debug)
    return sessions        
    

<b style="color:green">Testowanie:</b>

In [28]:
sessions = get_sessions(max_time_in_seconds=20, path_to_log_file="test.log")

for key, value in sessions.items():
    print(f"----------------------")
    print(f"IP address: {key}:")
    number_of_sessions = len(sessions[key])
    print(f"Number of sessions: {number_of_sessions}")
    for session in value:
        print(f"{session}")
    print(f"---------------------")
    print(f"\n")

----------------------
IP address: 66.194.6.77:
Number of sessions: 1
[['66.194.6.77', '05/Mar/2006:04:11:26 +0100', '/~annrec/slowniki/', '194']]
---------------------


----------------------
IP address: 83.14.59.98:
Number of sessions: 1
[['83.14.59.98', '06/Mar/2006:17:36:11 +0100', '/~stpublic/be6sem/lab02_03/', '200']]
---------------------


----------------------
IP address: 72.30.98.27:
Number of sessions: 1
[['72.30.98.27', '05/Mar/2006:04:13:39 +0100', '/~jacsta/e107/news.php?item.5', '200']]
---------------------


----------------------
IP address: 62.31.34.65:
Number of sessions: 2
[['62.31.34.65', '06/Mar/2006:17:36:10 +0100', '/~iwokar/blair_accessories/css/default.css', '200'], ['62.31.34.65', '06/Mar/2006:17:36:10 +0100', '/~iwokar/blair_accessories/jscript/funkcje.js', '200'], ['62.31.34.65', '06/Mar/2006:17:36:10 +0100', '/~iwokar/blair_accessories/imagenes/shared/logo_blair_1.gif', '200'], ['62.31.34.65', '06/Mar/2006:17:36:10 +0100', '/~iwokar/blair_accessories/im

<h1>Reguły asocjacyjne:</h1>

In [29]:
import pandas as pd
from apyori import apriori

<b>Testowanie dla Access Log Kinet 6</b>

In [30]:
max_time_seconds = 20 # 20 [s] w odstępie żadanie może wystąpić.
path_to_log_file = "access_log_kinet6_formatted.log" # Ścieżka do pliku z logami.
sessions = get_sessions(max_time_in_seconds=max_time_seconds, path_to_log_file=path_to_log_file) # Uzyskanie wszystkich sesji.

max_number_of_sessions = 0
ip_address_max_number_of_sessions = ""
for key, value in sessions.items():
    print(f"----------------------")
    print(f"IP address: {key}:")
    number_of_sessions = len(sessions[key])
    print(f"Number of sessions: {number_of_sessions}")
    for session in value:
        print(f"{session}")
    print(f"---------------------")
    print(f"\n")
    if max_number_of_sessions < number_of_sessions:
        max_number_of_sessions = number_of_sessions
        ip_address_max_number_of_sessions = key

print(f"Maksymalna ilość sesji: {max_number_of_sessions} oraz adres IP: {ip_address_max_number_of_sessions}\n")

----------------------
IP address: 62.141.215.90:
Number of sessions: 32
[['62.141.215.90', '05/Mar/2006:11:51:31 +0100', '/~adagot/p2p/', '141'], ['62.141.215.90', '05/Mar/2006:11:51:31 +0100', '/~adagot/p2p/style_p2p_index.css', '141'], ['62.141.215.90', '05/Mar/2006:11:51:32 +0100', '/~adagot/p2p/tlo_104.jpg', '141'], ['62.141.215.90', '05/Mar/2006:11:51:33 +0100', '/~adagot/p2p/menu_back.gif', '141'], ['62.141.215.90', '05/Mar/2006:11:51:34 +0100', '/~adagot/p2p/frames_free.gif', '141'], ['62.141.215.90', '05/Mar/2006:11:51:34 +0100', '/~adagot/p2p/poczta.gif', '141'], ['62.141.215.90', '05/Mar/2006:11:51:35 +0100', '/~adagot/p2p/ludzik-transparent.gif', '141']]
[['62.141.215.90', '05/Mar/2006:11:52:53 +0100', '/~adagot/p2p/download.htm', '141']]
[['62.141.215.90', '05/Mar/2006:11:52:53 +0100', '/~adagot/p2p/linia-pozioma.jpg', '141']]
[['62.141.215.90', '05/Mar/2006:12:06:42 +0100', '/~adagot/p2p/KLR0075.exe', '141']]
[['62.141.215.90', '05/Mar/2006:12:16:29 +0100', '/~adagot/p2p/

In [31]:
min_supports = [0.001, 0.002] #0.003, 0.004, 0.005]
min_confidence = [0.1, 0.2] #0.3, 0.4, 0.5]

max_time_seconds = 20 # 20 [s] w odstępie żadanie może wystąpić.
path_to_log_file = "access_log_kinet6_formatted.log" # Ścieżka do pliku z logami.

records = get_requests(path_to_log_file=path_to_log_file)

for min_support, min_confidence in zip(min_supports, min_confidence):
    print(f"\n---------------------------------------------------------")
    print(f"Minimum support: {min_support}, Minimum confidence: {min_confidence}\n")
    rules = apriori(
            records, min_support = min_support, min_confidence = min_confidence,
            min_lift = 2, min_length=2
    )
    results = pd.DataFrame(list(rules))
    print(f"1. Raw results: {results}\n")
    print(f"2. Formatted results: \n{results[['items', 'support']]}")
    print(f"\n3. Ordered statistics: \n{results['ordered_statistics']}")
    print(f"---------------------------------------------------------\n")


---------------------------------------------------------
Minimum support: 0.001, Minimum confidence: 0.1

1. Raw results:                                                  items   support  \
0                                             (153, /)  0.001113   
1                          (/%7Eirekm/list/-list, 153)  0.001546   
2               (/%7Eirekm/list/-list, 153.19.114.126)  0.001546   
3                                  (/favicon.ico, 206)  0.001237   
4                                  (/favicon.ico, 217)  0.005320   
..                                                 ...       ...   
565   (06/Mar/2006:11:10:08 +0100, 153, 153.19.105.64)  0.001485   
566  (06/Mar/2006:11:10:11 +0100, 172, 172.214.255.43)  0.001175   
567  (06/Mar/2006:11:10:12 +0100, 172, 172.214.255.43)  0.001485   
568   (158, 83.18.158.178, 06/Mar/2006:11:15:34 +0100)  0.001670   
569   (158, 83.18.158.178, 06/Mar/2006:11:21:46 +0100)  0.001237   

                                    ordered_statistics  
0 

----------------------

1. Dokonaj zgrubnej ceny danych zawartych w pliku access.log:

Odp.: Dane zawarte w pliku access.log zawierają wszystkie żądania dotyczące użytkowników, którzy manipulują zasobami.
Głównie są to operacje przy użyciu metody GET, czyli pobierania zasobu, a czasem POST (umieszczania zasosbu).

2. W jaki sposób zidentyfikowałeś sesję?

Odp.: Program na początku tworzy listę list/ macierz, która zawiera wszystkie żadania składające się z adresu IP, zasobu, daty/czasu oraz wyniku żądania (kod zwrócenia). Następnie program zbiera listę wszystkich adresów IP (oryginalne), potem dla każdego adresu IP znajduje jego listę żądań, finalnie tworzy sesję, do której doda następne żądanie jeżeli jest w określonym przedziale czasowy/ delta, jeżeli nie jest, to te żądanie przechodzi do nowej sesji. Innymi słowy dla każdego adresu IP tworzony słownik sesji, gdzie pod każdym ID są wskazane żądania należące do jednej wspólnej sesji. Parametr czasu określa przedział w jaki kolejne żądanie musi wystąpić np.: 5 [s].

3. Jaki czas przyjąłeś do identyfikacji sesji?

Odp.: Przykładowy czas sesji może wynosić 20 [s] lub więcej/ mniej, jest to parameter, który możemy modyfikować. W tej analizie czas sesji wynosił 20 [s].

4. Jak najlepiej oszacować moment, w którym sesja się zakończyła?

Odp.: Najlepiej oszacować moment drogą/ szeregiem eksperymentów, prób znalezienia najlepszej wartości, w której najbardziej wiarygodne informacje będą dla nas dostępne.

5. Jak potraktowałeś żądania pojedyncze (jednokrotne)?

Odp.: Żądanie pojedyczne zostały potraktowane jako jedna sesja składają się z jednego rekordu/ elementu. Co jest prawdą, ponieważ, żeby żądanie mogło istnieć to musi być nawiązana sesja.

6. Zastosuj mechanizm odkrywania reguł asocjacyjnych. Jakie reguły asocjacyjne uzyskałeś? Ile jest tych reguł?

Odp.: Mechanizm reguł asoscjacyjnych dla tak dużego zbioru danych może nie odkrywać najważniejszych reguł, bo jest ich zbyt mało. Najważniejsze reguły dotyczą obrazków np.: ikon do, których użytkownicy chcą mieć dostęp. 

7. Co możesz powiedzieć o tych regułach w kontekście budowy serwisu oraz użytkowania serwisu?

Odp.: Najważniejszym element analizowanej topologi jest rozmieszczenie najczęściej używanych elementów, którymi są obrazki. Użytkownicy próbują mieć dostęp do określonego obrazka i zawsze otrzymują tą samą odpowiedź, co może świadczyć np.: o braku uprawnień do zasobu.

8. Czy w Twoim zbiorze danych były wpisy robotów internetowych?

Odp.: Nie znaleziono wpisów robotów internetowych.

<h2>Statystki</h2>

10. Ile różnych żądań zostało zarejestrowanych w analizowanym pliku logs?
11. Skąd były najczęstsze żądania? Czy były to żądania od tej samej osoby?
12. Czy potrafisz zidentyfikować najczęściej odwiedzane strony podczas zidentyfikowanych sesji?
